In [1]:
import json
import os
import requests
from osgeo import gdal, ogr, osr
from datetime import datetime
osr.UseExceptions()  # Explicitly enable exceptions for GDAL methods (used to suppress warning)

In [2]:
# Create working directories
raster_folder = "./raster_files"
vector_folder = "./vector_files"
os.makedirs(raster_folder, exist_ok=True)
os.makedirs(vector_folder, exist_ok=True)

In [3]:
# Request user input
state = input("Enter a State:")
county = input("Enter a County (or leave blank):")
base_year = input("Choose a base year (2008-2022):")
spatial_reference = input("Enter a map projection (EPSG number):")
base_year = int(base_year)

Enter a State: Wisconsin
Enter a County (or leave blank): 
Choose a base year (2008-2022): 2022
Enter a map projection (EPSG number): 3857


In [4]:
def get_state_extent():
    
    state_API_endpoint="https://pdi.scinet.usda.gov/hosting/rest/services/Hosted/CONUS_States_Albers/FeatureServer/0/query?"
    parameters = f"where=STATE_NAME=%27{state}%27" \
                 f"&outSR={spatial_reference}" \
                  "&returnExtentOnly=true" \
                  "&f=geojson"
    
    request_URI = state_API_endpoint + parameters
    
    return(request_URI)

In [5]:
def get_county_extent():
    
    county_API_endpoint = "https://pdi.scinet.usda.gov/hosting/rest/services/Hosted/CONUS_Counties_Albers/FeatureServer/0/query?"
    parameters = f"where=NAME=%27{county}%27" \
                 f"%20AND%20STATE_NAME=%27{state}%27" \
                 f"&outSR={spatial_reference}" \
                  "&returnExtentOnly=true" \
                  "&f=geojson"
    
    request_URI = county_API_endpoint + parameters
    
    return(request_URI)

In [10]:
def get_state_boundary():
    
    state_API_endpoint="https://pdi.scinet.usda.gov/hosting/rest/services/Hosted/CONUS_States_Albers/FeatureServer/0/query?"
    parameters = f"where=STATE_NAME=%27{state}%27" \
                 f"&outSR={spatial_reference}" \
                  "&f=geojson"
    
    request_URI = state_API_endpoint + parameters

    return(request_URI)

In [11]:
def get_county_boundary():

    county_API_endpoint = "https://pdi.scinet.usda.gov/hosting/rest/services/Hosted/CONUS_Counties_Albers/FeatureServer/0/query?"
    parameters = f"where=NAME=%27{county}%27" \
                     f"%20AND%20STATE_NAME=%27{state}%27" \
                     f"&outSR={spatial_reference}" \
                      "&f=geojson"
    
    request_URI = county_API_endpoint + parameters

    return(request_URI)

In [8]:
# Converts year to epoch time on January 1, 20xx
def year_to_epoch(year):
    
    datetime_str = f"{year}-1-1 00:00:00.000"
    dt = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S.%f")
    epoch_time = int(dt.timestamp()*1000)
    
    return(epoch_time)

In [30]:
# Check if extent to be calculated is state-wide or county-wide
if not county:
    extent_URI = get_state_extent()
    boundary_URI = get_state_boundary()
    filename = f"{state}"
else:
    extent_URI = get_county_extent()
    boundary_URI = get_county_boundary()
    filename = f"{state}_{county}"

extent_response = requests.get(extent_URI)
boundary_response = requests.get(boundary_URI)

if extent_response.status_code == 200:
    URI_result = extent_response.json()
    extent = URI_result['extent']
    bounding_box = f"{extent['xmin']},{extent['ymin']},{extent['xmax']},{extent['ymax']}"
else:
    print(f"Unable to get data. Error code: {extent_response.status_code}")

if boundary_response.status_code == 200:
    geojson_data = json.loads(boundary_response.text)
else:
    print(f"Unable to get data. Error code: {boundary_response.status_code}")

-10340404.378432821,5234949.209224634,-9658537.320832817,5955285.108306542


In [26]:
# Get raster files
def get_raster_file(year):

    epoch_time = year_to_epoch(year)

    raster_API_endpoint = "https://pdi.scinet.usda.gov/image/rest/services/CDL_WM/ImageServer/exportImage?"
    parameters = f"bbox={bounding_box}" \
                 f"&bboxSR={spatial_reference}" \
                  "&size=4097,4097" \
                 f"&imageSR={spatial_reference}" \
                 f"&time={epoch_time}" \
                  "&format=tiff" \
                  "&f=image"
    
    URI = raster_API_endpoint + parameters
    response = requests.get(URI)
    
    if response.status_code == 200:
        raster_name = f"{filename}_{year}_EPSG{spatial_reference}.tiff"
        raster_path = f"{raster_folder}/{raster_name}"
        with open(raster_path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Unable to download the file. Error code: {response.status_code}")

    return(raster_path)

In [23]:
# Create outpput file
driver = ogr.GetDriverByName('ESRI Shapefile')

layer_name = f"{filename}_boundary"
vector_name = f"{filename}_boundary.shp"
vector_path = f"{vector_folder}/{vector_name}"
output_file = os.path.join(vector_folder, vector_name)

if (os.path.exists(output_file)):
    driver.DeleteDataSource(output_file)

# Create the data source
data_source = driver.CreateDataSource(output_file)

# Define spatial reference
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromEPSG(int(spatial_reference))

# Create the layer
layer = data_source.CreateLayer(layer_name, spatial_ref, ogr.wkbPolygon)

# Define a feature (with no attributes in this case)
feature_defn = layer.GetLayerDefn()

# Loop through geojson features
for feature in geojson_data['features']:
    # Get coordinates from the feature (this could be multipolygon)
    coordinates = feature["geometry"]["coordinates"]

    # Check if it is a multipolygon instance (a list)
    if (feature['geometry']['type'] == 'MultiPolygon'):
        # Handle multipolygon (coordinates is a list of polygons)
        for polygon_coords in coordinates:
            # Create LinearRing for each polygon
            ring = ogr.Geometry(ogr.wkbLinearRing)
            for coord in polygon_coords[0]:  # Assuming the coordinates are in [x, y] format
                ring.AddPoint(coord[0], coord[1])

            # Create Polygon geometry and add the ring
            polygon = ogr.Geometry(ogr.wkbPolygon)
            polygon.AddGeometry(ring)

            # Create a new feature for each polygon and set its geometry
            out_feature = ogr.Feature(feature_defn)
            out_feature.SetGeometry(polygon)

            # Add the feature to the layer
            layer.CreateFeature(out_feature)

            # Clear memory for the feature
            out_feature = None
    else:
        # Handle single polygon (coordinates is a single polygon)
        ring = ogr.Geometry(ogr.wkbLinearRing)
        for coord in coordinates[0]:
            ring.AddPoint(coord[0], coord[1])

        # Create Polygon geometry and add the ring
        polygon = ogr.Geometry(ogr.wkbPolygon)
        polygon.AddGeometry(ring)

        # Create a new feature for the polygon and set its geometry
        out_feature = ogr.Feature(feature_defn)
        out_feature.SetGeometry(polygon)

        # Add the feature to the layer
        layer.CreateFeature(out_feature)

        # Clear memory for the feature
        out_feature = None

# Clear memory for the data source after processing all features
data_source = None

In [29]:
# Download two file years to compare (base_year, base_year + 1)
base_raster = get_raster_file(base_year)
change_raster = get_raster_file(base_year+1)

https://pdi.scinet.usda.gov/image/rest/services/CDL_WM/ImageServer/exportImage?bbox=-10340404.378432821,5234949.209224634,-9658537.320832817,5955285.108306542&bboxSR=3857&size=4097,4097&imageSR=3857&time=1641016800000&format=tiff&f=image
https://pdi.scinet.usda.gov/image/rest/services/CDL_WM/ImageServer/exportImage?bbox=-10340404.378432821,5234949.209224634,-9658537.320832817,5955285.108306542&bboxSR=3857&size=4097,4097&imageSR=3857&time=1672552800000&format=tiff&f=image
